<a href="https://colab.research.google.com/github/sergiliesegangmaria/EMF-Aware-Power-Control-CF-mMIMO-vs-MC-mMIMO/blob/main/EMF_Aware_PowerControl_CF_mMIMO_vs_MC_mMIMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Readme**

The following numerical simulations evaluate the data rate performance of heuristic power control methods in cell-free (CF) and multi-cell (MC) mMIMO networks for DL and UL settings. The results of EMF exposure, also known as perceived user radiation, measured in terms of incident power density (DL) and specific absorption rate (UL), are also provided.

To execute the associated code, please run first all the cells in section *General* and later those of the desired scenario (*DL* or *UL*).

# **General**

## **Package Import**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd

##**Orthogonal Pilots**

In [ ]:
def PilotSetOrthogonal(tau_p):
    Pilot_Set = np.zeros((tau_p, tau_p))
    A = np.random.randn(tau_p, tau_p)
    A_orth = scipy.linalg.orth(A)

    for j in range(tau_p):
        Pilot_Set[:, j] = A_orth[:, j]

    return Pilot_Set

##**Random Pilot Assignment**

In [ ]:
def PilotAssignmentRandom(K, tau_p, Pilot_Set):
    val_reuse = K // tau_p
    Assignment = np.zeros(val_reuse * tau_p)
    Phi = np.zeros((tau_p, val_reuse * tau_p))

    for i in range(val_reuse):
        start_col = i * tau_p
        end_col = (i + 1) * tau_p
        Phi[:, start_col:end_col] = Pilot_Set
        Assignment[start_col:end_col] = np.arange(1, tau_p + 1)

    val_remain = K - val_reuse * tau_p
    Phi = np.concatenate((Phi, Pilot_Set[:, :val_remain]), axis=1)
    Assignment = np.concatenate((Assignment, np.arange(1, val_remain + 1)))

    return Phi, Assignment

##**Guarantee Minimum Distance**

In [ ]:
def GuaranteeMinimumDistance(pos, user_pos, D, M, K, d_min, wrapped):
    if wrapped:
        for k in range(K):
            d_2D = np.zeros(M)
            for m in range(M):
                d_x = min(abs(pos[m, 0] - user_pos[k, 0]), D - abs(pos[m, 0] - user_pos[k, 0]))
                d_y = min(abs(pos[m, 1] - user_pos[k, 1]), D - abs(pos[m, 1] - user_pos[k, 1]))
                d_2D[m] = np.sqrt(d_x**2 + d_y**2)

            while np.any(d_2D < d_min):
                user_pos[k, :2] = np.random.rand(1, 2) * D
                for m in range(M):
                    d_x = min(abs(pos[m, 0] - user_pos[k, 0]), D - abs(pos[m, 0] - user_pos[k, 0]))
                    d_y = min(abs(pos[m, 1] - user_pos[k, 1]), D - abs(pos[m, 1] - user_pos[k, 1]))
                    d_2D[m] = np.sqrt(d_x**2 + d_y**2)
    else:
        for k in range(K):
            d_2D = np.zeros(M)
            for m in range(M):
                d_x = user_pos[k, 0] - pos[m, 0]
                d_y = user_pos[k, 1] - pos[m, 1]
                d_2D[m] = np.sqrt(d_x**2 + d_y**2)

            while np.any(d_2D < d_min):
                user_pos[k, :2] = np.random.rand(1, 2) * D
                for m in range(M):
                    d_x = user_pos[k, 0] - pos[m, 0]
                    d_y = user_pos[k, 1] - pos[m, 1]
                    d_2D[m] = np.sqrt(d_x**2 + d_y**2)
    return user_pos


## **Channel Model**

In [ ]:
def ChannelModel(f, pos, user_pos, N, M, K, antenna_spacing, wavelength, wrapped, D, d_min, fading):
    H = np.zeros((N, M, K), dtype=complex)
    Beta = np.zeros((M, K))
    C = np.zeros((N, N, M, K), dtype=complex)

    if fading == 2:
        F_LoS = ShadowingCorrelationModel(M, K, user_pos, wrapped, D, 1)
    else:
        F_LoS = np.zeros((M, K))

    F_NLoS = ShadowingCorrelationModel(M, K, user_pos, wrapped, D, 0)

    f_GH = f * 1e-9
    d0 = 18
    kappa = 2 * np.pi / wavelength
    S = kappa * antenna_spacing

    for m in range(M):
        for k in range(K):
            if wrapped:
                d_x = min(np.mod(pos[m, 0] - user_pos[k, 0], D), np.mod(user_pos[k, 0] - pos[m, 0], D))
                d_y = min(np.mod(pos[m, 1] - user_pos[k, 1], D), np.mod(user_pos[k, 1] - pos[m, 1], D))
            else:
                d_x = user_pos[k, 0] - pos[m, 0]
                d_y = user_pos[k, 1] - pos[m, 1]
            d_z = user_pos[k, 2] - pos[m, 2]
            d_2D = np.sqrt(d_x**2 + d_y**2)
            d_3D = np.sqrt(d_x**2 + d_y**2 + d_z**2)

            if fading == 2:
                Prob_LoS = min(1, d0 / d_2D + (1 - d0 / d_2D) * np.exp(-d_2D / (2 * d0)))
            else:
                Prob_LoS = 0

            if Prob_LoS == 1:
                R = np.finfo(float).max
            else:
                R = Prob_LoS / (1 - Prob_LoS)

            if wrapped:
                extra_dist = np.zeros(N)
                for n in range(N):
                    d_y_aux = min(np.mod(pos[m, 1] + antenna_spacing * n - user_pos[k, 1], D), np.mod(user_pos[k, 1] - pos[m, 1] + antenna_spacing * n, D)
                    )
                    extra_dist[n] = d_3D - np.sqrt(d_x**2 + d_y_aux**2 + d_z**2)
            else:
                azimuth = np.arctan2(d_y, d_x)
                elevation = np.arctan(d_z / np.sqrt(d_x**2 + d_y**2))
                aoa = [azimuth, elevation]
                a, e, r = np.cart2sph(d_x, d_y, d_z)
                if abs(r - d_3D) > 1e-3 or abs(a - azimuth) > 1e-3 or abs(e - elevation) > 1e-3:
                    raise ValueError('Badly defined Cartesian coordinates')

            d_2D = np.sqrt(d_x**2 + d_y**2)
            d_BP = 4 * (pos[m, 2] - 1) * (user_pos[k, 2] - 1) / wavelength
            if d_2D < d_BP:
                PL_LoS = 22 * np.log10(d_3D) + 28 + 20 * np.log10(f_GH)
            else:
                PL_LoS = 40 * np.log10(d_3D) + 28 + 20 * np.log10(f_GH) - 9 * np.log10(d_BP**2 + (pos[m, 2] - user_pos[k, 2])**2)
            Beta_mk_dB = -PL_LoS + F_LoS[m, k]

            if d_2D < d_min:
                raise ValueError('Minimum distance not complied!')

            if Prob_LoS == 0:
                PL_NLoS = 36.7 * np.log10(d_3D) + 22.7 + 26 * np.log10(f_GH) - 0.3 * (user_pos[k, 2] - 1.5)
                PL_NLoS = max(PL_LoS, PL_NLoS)
                Beta_mk_dB = -PL_NLoS + F_NLoS[m, k]
            Beta_mk = 10**(Beta_mk_dB / 10)
            Beta[m, k] = Beta_mk
            H_NLoS = np.sqrt(1/2) * (np.random.randn(N) + 1j * np.random.randn(N))

            if wrapped:
                H_LoS = np.exp(-1j * kappa * extra_dist)
            else:
                H_LoS = np.exp(1j * S * np.arange(0, N) * np.cos(aoa[1]) * np.sin(aoa[0]))

            P_off = np.exp(1j * 2 * np.pi * np.random.rand(1))
            H[:, m, k] = np.sqrt(Beta_mk / (1 + R)) * (np.sqrt(R) * H_LoS * P_off + H_NLoS)
            C[:, :, m, k] = np.sqrt(Beta_mk / (1 + R)) * (np.sqrt(R) * np.outer(H_LoS, H_LoS.conj()) + np.eye(N))

    return H, Beta, C

### **Shadowing Correlation Model**

In [ ]:
def ShadowingCorrelationModel(M, K, user_pos, wrapped, D, Prob_LoS):
    R_F = np.zeros((M * K, M * K))

    for m in range(M):
        R_temp = np.zeros((K, K))
        for k in range(K):
            for i in range(K):
                if wrapped:
                    d_x = min(np.mod(user_pos[k, 0] - user_pos[i, 0], D), np.mod(user_pos[i, 0] - user_pos[k, 0], D))
                    d_y = min(np.mod(user_pos[k, 1] - user_pos[i, 1], D), np.mod(user_pos[i, 1] - user_pos[k, 1], D))
                else:
                    d_x = abs(user_pos[k, 0] - user_pos[i, 0])
                    d_y = abs(user_pos[k, 1] - user_pos[i, 1])
                d_z = abs(user_pos[k, 2] - user_pos[i, 2])
                d_k_i = np.sqrt(d_x**2 + d_y**2 + d_z**2)

                if Prob_LoS == 0:
                    std_shadowing = 4
                    R_temp[k, i] = (std_shadowing**2) * np.exp(-d_k_i / 13)
                else:
                    std_shadowing = 3
                    R_temp[k, i] = (std_shadowing**2) * np.exp(-d_k_i / 10)
        R_F[m * K:(m + 1) * K, m * K:(m + 1) * K] = R_temp

    v = np.random.randn(M * K)
    _, Lambda, V = np.linalg.svd(R_F)
    F_vec = np.atleast_1d(V @ np.sqrt(np.diag(Lambda)) @ v)
    F = np.zeros((M, K))

    for m in range(M):
        F[m, :] = F_vec[m * K:(m + 1) * K]

    return F


## **Linear MMSE Channel Estimation**

In [ ]:
def ChannelEstimation(N, M, K, tau_p, H, C, Phi, Eta_pilot, noise_variance):
    H_estimate = np.zeros((N, M, K), dtype=complex)
    Beta = np.zeros((M, K))

    if np.array_equal(C[:, :, 0, 0], np.diag(np.diag(C[:, :, 0, 0]))):
        for m in range(M):
            Ym = np.sqrt(noise_variance / 2) * (np.random.randn(N, tau_p) + 1j * np.random.randn(N, tau_p))
            for k in range(K):
                Ym = Ym + np.sqrt(Eta_pilot[k]) * np.outer(H[:, m, k], Phi[:, k].conj())
            for k in range(K):
                y_mk = np.mult(Ym, Phi[:, k])
                sum_alpha = 0
                for j in range(K):
                    Beta[m, j] = C[0, 0, m, j]
                    sum_alpha = sum_alpha + Eta_pilot[j] * Beta[m, j] * np.abs(np.inner(Phi[:, j].conj(), Phi[:, k]))**2
                alpha_mk = np.sqrt(Eta_pilot[k]) * Beta[m, k] / (sum_alpha + noise_variance)
                H_estimate[:, m, k] = alpha_mk * y_mk

    else:
        for m in range(M):
            Ym = np.sqrt(noise_variance / 2) * (np.random.randn(N, tau_p) + 1j * np.random.randn(N, tau_p))
            for k in range(K):
                Ym = Ym + np.sqrt(Eta_pilot[k]) * np.outer(H[:, m, k], Phi[:, k].conj())
            for k in range(K):
                y_mk = np.matmul(Ym, Phi[:, k])
                sum_alpha = 0
                for j in range(K):
                    sum_alpha = sum_alpha + Eta_pilot[j] * C[:, :, m, j] * np.abs(np.inner(Phi[:, j].conj(), Phi[:, k]))**2
                alpha_mk = np.sqrt(Eta_pilot[k]) * np.matmul(C[:, :, m, k], np.linalg.inv(sum_alpha + noise_variance * np.eye(N)))
                H_estimate[:, m, k] = np.matmul(alpha_mk, y_mk)

    return H_estimate

## **User Association UC-CF-mMIMO**

In [ ]:
def UserAssociationUC(M, K, Beta, N):
    M_ASS = []

    for k in range(K):
        Beta_k = Beta[:, k]
        index = np.argsort(-Beta_k)
        M_ASS.append(sorted(index[:min(N,M)]))

    K_ASS = []

    for m in range(M):
        K_m = []
        for k in range(K):
            M_k = M_ASS[k]
            if m in M_k:
                K_m.append(k)
        K_ASS.append(sorted(K_m))

    return K_ASS, M_ASS

## **User Association APC-CF-mMIMO**

In [ ]:
def UserAssociationAPC(M, K, Beta, N):
    K_ASS = []

    for m in range(M):
        Beta_m = Beta[m, :]
        index = np.argsort(-Beta_m)
        K_ASS.append(sorted(index[:min(N,K)]))

    M_ASS = []

    for k in range(K):
        M_k = []
        for m in range(M):
            K_m = K_ASS[m]
            if k in K_m:
                M_k.append(m)
        M_ASS.append(sorted(M_k))

    return K_ASS, M_ASS

# **DL**

## **MRT Beamformer**

In [ ]:
def Precoding_DL(N, M, K, H_estimate):
    W = np.zeros((N, M, K), dtype=complex)
    Gamma = np.zeros((M, K))

    for k in range(K):
        for m in range(M):
            W[:, m, k] = H_estimate[:, m, k]
            Gamma[m, k] = np.linalg.norm(W[:, m, k]) ** 2

    return W, Gamma

## **Power Control**

In [ ]:
def PowerControl_DL(M, K, N_power, Gamma, Pt, K_ASS, M_ASS, EMF, H, W, noise_variance):
    IPD = np.zeros((K, N_power))
    R = np.zeros((K, N_power))

    Eta, IPD[:, 0] = UniformPowerControl_DL(M, K, Gamma, Pt, K_ASS, M_ASS, EMF, H, W)
    R[:, 0] = DataRate_DL(K, Eta, H, W, noise_variance, M_ASS)

    Eta, IPD[:, 1] = ProportionalPowerControl_DL(M, K, Gamma, Pt, K_ASS, M_ASS, EMF, H, W)
    R[:, 1] = DataRate_DL(K, Eta, H, W, noise_variance, M_ASS)

    return R, IPD

### **Uniform Power Control**

In [ ]:
def UniformPowerControl_DL(M, K, Gamma, Pt, K_ASS, M_ASS, EMF, H, W):
    Eta = np.zeros((M, K))
    IPD = np.zeros(K)
    constraint = False
    counter = 0
    step = 0.9

    while not constraint:
        counter += 1
        for m in range(M):
            K_m = K_ASS[m]
            for k in range(K):
                if k in K_m:
                    Eta[m, k] = Pt / (Gamma[m, k] * len(K_m))

        IPD = IncidentPowerDensity_DL(K, M_ASS, Eta, H, W)

        if np.all(IPD <= EMF):
            constraint = True
        else:
            Pt = Pt * step**(1 / counter)

        if counter == 1e3:
            counter = 0
            step -= 0.1

    CheckPowerConstraint_DL(M, Eta, Gamma, Pt, True)

    return Eta, IPD

### **Proportional Power Control**

In [ ]:
def ProportionalPowerControl_DL(M, K, Gamma, Pt, K_ASS, M_ASS, EMF, H, W):
    Eta = np.zeros((M, K))
    IPD = np.zeros(K)
    constraint = False
    counter = 0
    step = 0.9

    while not constraint:
        counter += 1
        for m in range(M):
            K_m = K_ASS[m]
            if K_m:
                sum_gamma = np.sum([Gamma[m, k] for k in K_m])
                for k in K_m:
                    Eta[m, k] = Pt / sum_gamma

        IPD = IncidentPowerDensity_DL(K, M_ASS, Eta, H, W)

        if np.all(IPD <= EMF):
            constraint = True
        else:
            Pt = Pt * step**(1 / counter)

        if counter == 1e3:
            counter = 0
            step -= 0.1

    CheckPowerConstraint_DL(M, Eta, Gamma, Pt, True)

    return Eta, IPD

### **Data Rate**

In [ ]:
def DataRate_DL(K, Eta, H, W, noise_variance, M_ASS):
    SE = np.zeros(K)

    for k in range(K):
        M_k = M_ASS[k]
        if M_k:
            sum_num = 0
            for m_index in range(len(M_k)):
                m = M_k[m_index]
                sum_num += np.sqrt(Eta[m, k]) * H[:, m, k].conj().T @ W[:, m, k]
            num_k = abs(sum_num)**2

            den_k = 0
            for j in range(K):
                sum_den = 0
                M_j = M_ASS[j]

                if j != k:
                    for m_index in range(len(M_j)):
                        m = M_j[m_index]
                        sum_den += np.sqrt(Eta[m, j]) * H[:, m, k].conj().T @ W[:, m, j]

                    den_k += abs(sum_den)**2

            SE[k] = np.log2(1 + num_k / (den_k + noise_variance))
        else:
            SE[k] = 0

    return np.real(SE)

### **Incident Power Density**

In [ ]:
def IncidentPowerDensity_DL(K, M_ASS, Eta, H, W):
    IPD = np.zeros(K)

    for k in range(K):
        for j in range(K):
            aux = 0
            M_j = M_ASS[j]

            for m_index in range(len(M_j)):
                m = M_j[m_index]
                aux += np.sqrt(Eta[m, j]) * H[:, m, k].conj().T @ W[:, m, j]

            IPD[k] += abs(aux)**2

    return IPD

### **Check Power Constraint**

In [ ]:
def CheckPowerConstraint_DL(M, Eta, Gamma, Pt, equality):
    for m in range(M):
        power_AP_m = np.sum(Eta[m, :] * Gamma[m, :])
        if power_AP_m != 0:
            if equality:
                if abs(power_AP_m - Pt) / Pt > 1e-3:
                    raise ValueError('Power constraint not satisfied with equality')
            else:
                if power_AP_m > Pt:
                    raise ValueError('Power constraint not satisfied with inequality')

## **Main**

In [ ]:
np.random.seed(1)                                                               # For reproducibility

# Simulation options
wrapped = True                                                                  # Wrapped channel model
fading = 2                                                                      # Fading: Rayleigh (1) or Rice (2)
CSI = 2                                                                         # CSI assumption: perfect (1) or imperfect (2)
constraint = 1                                                                  # EMF constraint: inactive (0) or active (1)

# Optimization parameters
N_channels = int(1e3)                                                           # Number of channel realizations
N_power = 2                                                                     # Number of power control policies: uniform (UPC) and proportional (PPC)

# Channel generation
M_CF = 50                                                                       # Number of access points (APs) in the cell-free massive MIMO (CF-mMIMO) setup
N_AP = 4                                                                        # Number of antennas at the AP
N_BS = M_CF                                                                     # Number of antennas for the multi-cell mMIMO (MC-mMIMO) base stations (BSs)
M_MC = N_AP                                                                     # Number of BSs in the MC-mMIMO case
K = 40                                                                          # Number of users
N_users_APC = 10                                                                # Number of users that each AP is connected to in the AP-centric CF-mMIMO (APC-CF-mMIMO) setup
N_APs_UC = 5                                                                    # Number of APs to which each user is connected to in the user-centric CF-mMIMO (UC-CF-mMIMO) setup
f = 1.9e9                                                                       # Carrier frequency
wavelength = 3e8 / f                                                            # Wavelength
antenna_spacing = wavelength / 2                                                # Antenna spacing (uniform linear array)
BW = 20e6                                                                       # Bandwidth
h_AP = 10                                                                       # AP/BS height
h_user = 1.65                                                                   # User height
d_min = 1e1                                                                     # Minimum 2D AP/BS-user distance
D = 1e3                                                                         # D x D is the size of the considered square area
antenna_gain = 1                                                                # Antenna gain of AP/BS/users

# Power budgets
P_AP = 200e-3                                                                   # Power budget of the single AP (CF-mMIMO setup)
P_AP = P_AP * antenna_gain                                                      # ... add antenna gain
P_BS = P_AP * M_CF / M_MC                                                       # Power budget of the BS (MC-mMIMO setup)

# Noise variance
noise_figure = 9                                                                # Noise figure in dB
N0 = -174                                                                       # PSD noise in dBm/Hz (AWGN)
noise_variance = BW * 10**(0.1 * noise_figure) * 10**(0.1 * N0) * 1e-3          # F*N0*B

# Pilot assignment
tau_c = 200                                                                     # Length of coherence interval in time/frequency samples
tau_p = int(np.ceil(K / 2))                                                     # Number of pilot symbols
Pilot_Set = PilotSetOrthogonal(tau_p)                                           # Set of orthogonal pilots
Phi, Assignment = PilotAssignmentRandom(K, tau_p, Pilot_Set)                    # Random pilot assignment
P_training = 100e-3                                                             # Users (UL) training power
P_training *= antenna_gain                                                      # ... add antenna gain
Eta_pilot = tau_p * P_training * np.ones(K)                                     # ... over tau_p sequences

# Pure CF (or fully-connected network) associations
K_ASS_CF = [list(range(0, K)) for _ in range(M_CF)]                             # All users are associated to all APs
M_ASS_CF = [list(range(0, M_CF)) for _ in range(K)]                             # All APs are associated to all users

# Data rates and incident power densities (IPDs)
R_CF = np.zeros((K, N_power, N_channels))
R_UC = np.zeros((K, N_power, N_channels))
R_APC = np.zeros((K, N_power, N_channels))
R_MC = np.zeros((K, N_power, N_channels))

I_CF = np.zeros((K, N_power, N_channels))
I_UC = np.zeros((K, N_power, N_channels))
I_APC = np.zeros((K, N_power, N_channels))
I_MC = np.zeros((K, N_power, N_channels))

# Electromagnetic field (EMF) exposure DL constraints
EMF = 10                                                                        # Maximum IPD in W/m2
EMF = EMF * wavelength**2 / (4 * np.pi)                                         # Rescale to include wavelength and 4*pi (units of Watts)

if constraint == 0:
    EMF = np.inf

# Monte Carlo iterations
for n in range(0, N_channels):
    print(f'Channel realization {n + 1} out of {N_channels}')

    # User and AP positions
    AP_pos = np.concatenate((D * np.random.rand(M_CF, 2), h_AP * np.ones((M_CF, 1))), axis=1)
    user_pos = np.concatenate((D * np.random.rand(K, 2), h_user * np.ones((K, 1))), axis=1)

    # 4 fixed BSs with N_AP antenna each and located at the center of the 4 quadrants of the deployment area.
    BI_pos = np.zeros((M_MC, 3))
    BI_pos[0, :] = [D / 4, D / 4, h_AP]
    BI_pos[1, :] = [3 * D / 4, D / 4, h_AP]
    BI_pos[2, :] = [3 * D / 4, 3 * D / 4, h_AP]
    BI_pos[3, :] = [D / 4, 3 * D / 4, h_AP]

    # Guarantee minimum distance
    user_pos = GuaranteeMinimumDistance(np.concatenate((AP_pos, BI_pos), axis=0), user_pos, D, M_CF + M_MC, K, d_min, wrapped)

    if n == 0:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(user_pos[:, 0], user_pos[:, 1], user_pos[:, 2], marker='x')
        ax.scatter(AP_pos[:, 0], AP_pos[:, 1], AP_pos[:, 2], marker='o')
        ax.scatter(BI_pos[:, 0], BI_pos[:, 1], BI_pos[:, 2], marker='s')
        ax.set_xlabel('X-axis', labelpad=10)
        ax.set_ylabel('Y-axis', labelpad=10)
        ax.set_title('Deployment Example', fontsize=15)
        ax.legend(['User', 'AP', 'BS'], loc='best')
        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        ax.minorticks_on()
        plt.show()

    # Scenarios CF-mMIMO
    # Channel and large-scale fading coefficients
    H, Beta, C = ChannelModel(f, AP_pos, user_pos, N_AP, M_CF, K, antenna_spacing, wavelength, wrapped, D, d_min, fading)

    # LMMSE estimation
    if CSI == 1:
        H_estimate = H
    elif CSI == 2:
        H_estimate = ChannelEstimation(N_AP, M_CF, K, tau_p, H, C, Phi, Eta_pilot, noise_variance)

    # MRT beamformer
    W, Gamma = Precoding_DL(N_AP, M_CF, K, H_estimate)

    # I) Pure CF
    # Associations
    K_ASS = K_ASS_CF
    M_ASS = M_ASS_CF

    # Power control results
    R, I = PowerControl_DL(M_CF, K, N_power, Gamma, P_AP, K_ASS, M_ASS, EMF, H, W, noise_variance)
    R_CF[:, :, n] = R
    I_CF[:, :, n] = I

    # II) UC
    # Associations
    K_ASS, M_ASS = UserAssociationUC(M_CF, K, Beta, N_APs_UC)

    # Power control results
    R, I = PowerControl_DL(M_CF, K, N_power, Gamma, P_AP, K_ASS, M_ASS, EMF, H, W, noise_variance)
    R_UC[:, :, n] = R
    I_UC[:, :, n] = I

    # III) APC
    # Associations
    K_ASS, M_ASS = UserAssociationAPC(M_CF, K, Beta, N_users_APC)

    # Power control results
    R, I = PowerControl_DL(M_CF, K, N_power, Gamma, P_AP, K_ASS, M_ASS, EMF, H, W, noise_variance)
    R_APC[:, :, n] = R
    I_APC[:, :, n] = I

    # Scenario MC-mMIMO
    # Channel and large-scale fading coefficients
    H, Beta, C = ChannelModel(f, BI_pos, user_pos, N_BS, M_MC, K, antenna_spacing, wavelength, wrapped, D, d_min, fading)

    # Channel estimation
    if CSI == 1:
        H_estimate = H
    elif CSI == 2:
        H_estimate = ChannelEstimation(N_BS, M_MC, K, tau_p, H, C, Phi, Eta_pilot, noise_variance)

    # MRT beamformer
    W, Gamma = Precoding_DL(N_BS, M_MC, K, H_estimate)

    # User association MC-mMIMO
    K_ASS, M_ASS = UserAssociationUC(M_MC, K, Beta, 1)

    # Power control results
    R, I = PowerControl_DL(M_MC, K, N_power, Gamma, P_BS, K_ASS, M_ASS, EMF, H, W, noise_variance)
    R_MC[:, :, n] = R
    I_MC[:, :, n] = I

# Convert rates from bps/Hz to bps
R_CF *= BW * (1 - tau_p / tau_c) / 2
R_UC *= BW * (1 - tau_p / tau_c) / 2
R_APC *= BW * (1 - tau_p / tau_c) / 2
R_MC *= BW * (1 - tau_p / tau_c) / 2

I_CF *= 4 * np.pi / (wavelength ** 2)
I_UC *= 4 * np.pi / (wavelength ** 2)
I_APC *= 4 * np.pi / (wavelength ** 2)
I_MC *= 4 * np.pi / (wavelength ** 2)

# Plot rate results
plt.figure()
markers = ["o","s"]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
for i in range(N_power):
      ind = 0
      aux_R = R_CF[:, i, :]
      ecdf = scipy.stats.ecdf(aux_R.ravel() / 1e6)
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 40))

for i in range(N_power):
      ind = 1
      aux_R = R_UC[:, i, :]
      ecdf = scipy.stats.ecdf(aux_R.ravel() / 1e6)
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 20))

for i in range(N_power):
      ind = 2
      aux_R = R_APC[:, i, :]
      ecdf = scipy.stats.ecdf(aux_R.ravel() / 1e6)
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 10))

for i in range(N_power):
      ind = 3
      aux_R = R_MC[:, i, :]
      ecdf = scipy.stats.ecdf(aux_R.ravel() / 1e6)
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 5))

plt.xlabel('Rate per user [Mbit/s]', fontsize=15)
plt.ylabel('CDF', fontsize=15)
plt.title('Throughput vs Power Control (DL)', fontsize=15)
plt.legend(['CF-mMIMO: UPC', 'CF-mMIMO: PPC', 'UC-CF-mMIMO: UPC', 'UC-CF-mMIMO: PPC', 'APC-CF-mMIMO: UPC',
            'APC-CF-mMIMO: PPC', 'MC-mMIMO: UPC', 'MC-mMIMO: PPC'], loc='best', fontsize=15)
plt.grid()
plt.show()

# Plot IPD results
plt.figure()
for i in range(N_power):
      ind = 0
      aux_I = I_CF[:, i, :]
      ecdf = scipy.stats.ecdf(aux_I.ravel())
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 40))

for i in range(N_power):
      ind = 1
      aux_I = I_UC[:, i, :]
      ecdf = scipy.stats.ecdf(aux_I.ravel())
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 20))

for i in range(N_power):
      ind = 2
      aux_I = I_APC[:, i, :]
      ecdf = scipy.stats.ecdf(aux_I.ravel())
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 10))

for i in range(N_power):
      ind = 3
      aux_I = I_MC[:, i, :]
      ecdf = scipy.stats.ecdf(aux_I.ravel())
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 5))

plt.xlabel('IPD per user [W/m$^2$]', fontsize=15)
plt.ylabel('CDF', fontsize=15)
plt.title('Exposure vs Power Control (DL)', fontsize=15)
plt.legend(['CF-mMIMO: UPC', 'CF-mMIMO: PPC', 'UC-CF-mMIMO: UPC', 'UC-CF-mMIMO: PPC', 'APC-CF-mMIMO: UPC',
            'APC-CF-mMIMO: PPC', 'MC-mMIMO: UPC', 'MC-mMIMO: PPC'], loc='best', fontsize=15)
plt.xscale('log')
plt.grid()
plt.show()

# **UL**

## **MRC Equalizer**

In [ ]:
def Filtering_UL(N, M, K, H_estimate):
    G = np.zeros((N, M, K), dtype=complex)

    for k in range(K):
        for m in range(M):
            G[:, m, k] = H_estimate[:, m, k]

    return G

## **Power Control**

In [ ]:
def PowerControl_UL(K, N_power, C, P_user, M_ASS, b, EMF, H, G, noise_variance, P_0, alpha):
    SAR = np.zeros((K, N_power))
    R = np.zeros((K, N_power))

    Eta, SAR[:, 0] = UniformPowerControl_UL(K, P_user, b, EMF)
    R[:, 0] = DataRate_UL(K, Eta, H, G, noise_variance, M_ASS)

    Eta, SAR[:, 1] = FractionalPowerControl_UL(K, C, M_ASS, b, EMF, P_user, P_0, alpha)
    R[:, 1] = DataRate_UL(K, Eta, H, G, noise_variance, M_ASS)

    return R, SAR

### **Uniform Power Control**

In [ ]:
def UniformPowerControl_UL(K, P_user, b, EMF):
    constraint = False
    counter = 0
    step = 0.9

    while not constraint:
        counter += 1
        Eta = P_user * np.ones(K)
        SAR = SpecificAbsorptionRate_UL(b, Eta)

        if np.all(SAR <= EMF):
            constraint = True
        else:
            P_user = P_user * (step ** (1 / counter))

        if counter == 1e3:
            counter = 0
            step -= 0.1

    CheckPowerConstraint_UL(K, Eta, P_user, True)

    return Eta, SAR

### **Fractional Power Control**

In [ ]:
def FractionalPowerControl_UL(K, C, M_ASS, b, EMF, P_user, P_0, alpha):
    Eta = np.zeros(K)
    constraint = False
    counter = 0
    step = 0.9

    while not constraint:
        counter += 1
        for k in range(K):
            M_k = M_ASS[k]
            if M_k:
                aux = 0
                for i in range(0, len(M_k)):
                    m = M_k[i]
                    aux += np.real(np.trace(C[:, :, m, k])) ** 2
                xi_k = np.sqrt(aux)
                Eta[k] = min(P_user, P_0 * xi_k ** (-alpha))
            else:
                Eta[k] = 0
        SAR = SpecificAbsorptionRate_UL(b, Eta)
        if np.all(SAR <= EMF):
            constraint = True
        else:
            P_user = P_user * step ** (1 / counter)

        if counter == 1e3:
            counter = 0
            step -= 0.1

    CheckPowerConstraint_UL(K, Eta, P_user, False)

    return Eta, SAR


### **Data Rate**

In [ ]:
def DataRate_UL(K, Eta, H, G, noise_variance, M_ASS):
    SE = np.zeros(K)

    for k in range(K):
        M_k = M_ASS[k]
        if M_k:
            sum_num = 0
            for m_index in range(0, len(M_k)):
                m = M_k[m_index]
                sum_num += np.inner(G[:, m, k].conj(), H[:, m, k])

            num_k = Eta[k] * np.abs(sum_num) ** 2

            den_k = 0
            for j in range(K):
                if j != k:
                    M_j = M_ASS[j]
                    sum_den = 0
                    for m_index in range(0, len(M_j)):
                        m = M_j[m_index]
                        sum_den += np.inner(G[:, m, k].conj(), H[:, m, j])
                    den_k += Eta[j] * np.abs(sum_den) ** 2

            sum_noise = 0
            for m_index in range(0, len(M_k)):
                m = M_k[m_index]
                sum_noise += noise_variance * np.linalg.norm(G[:, m, k]) ** 2

            SE[k] = np.log2(1 + num_k / (den_k + sum_noise))
        else:
            SE[k] = 0

    return SE

### **Specific Absorption Rate**

In [ ]:
def SpecificAbsorptionRate_UL(b, Eta):
    SAR = b * Eta
    return SAR

### **Check Power Constraint**

In [ ]:
def CheckPowerConstraint_UL(K, Eta, P_user, equality):
    for k in range(K):
        power_user_k = Eta[k]
        if power_user_k != 0:
            if equality:
                if abs(power_user_k - P_user) / P_user > 1e-3:
                    raise ValueError('Power constraint not satisfied with equality')
            else:
                if power_user_k > P_user:
                    raise ValueError('Power constraint not satisfied with inequality')

## **Main**

In [ ]:
np.random.seed(1)                                                               # For reproducibility

# Simulation options
wrapped = True                                                                  # Wrapped channel model
fading = 2                                                                      # Fading: Rayleigh (1) or Rice (2)
CSI = 2                                                                         # CSI assumption: perfect (1) or imperfect (2)
constraint = 1                                                                  # EMF constraint: inactive (0) or active (1)

# Optimization parameters
N_channels = int(1e3)                                                           # Number of channel realizations
N_power = 2                                                                     # Number of power control policies: uniform (UPC) and fractional (FPC)

# Channel generation
M_CF = 50                                                                       # Number of access points (APs) in the cell-free massive MIMO (CF-mMIMO) setup
N_AP = 4                                                                        # Number of antennas at the AP
N_BS = M_CF                                                                     # Number of antennas for the multi-cell mMIMO (MC-mMIMO) base stations (BSs)
M_MC = N_AP                                                                     # Number of BSs in the MC-mMIMO case
K = 40                                                                          # Number of users
N_users_APC = 10                                                                # Number of users that each AP is connected to in the AP-centric CF-mMIMO (APC-CF-mMIMO) setup
N_APs_UC = 5                                                                    # Number of APs to which each user is connected to in the user-centric CF-mMIMO (UC-CF-mMIMO) setup
f = 1.9e9                                                                       # Carrier frequency
wavelength = 3e8 / f                                                            # Wavelength
antenna_spacing = wavelength / 2                                                # Antenna spacing (uniform linear array)
BW = 20e6                                                                       # Bandwidth
h_AP = 10                                                                       # AP/BS height
h_user = 1.65                                                                   # User height
d_min = 1e1                                                                     # Minimum 2D AP/BS-user distance
D = 1e3                                                                         # D x D is the size of the considered square area
antenna_gain = 1                                                                # Antenna gain of users

# Power budgets
P_user = 100e-3                                                                 # Power budget of the single user (CF-mMIMO setup)
P_user *= antenna_gain                                                          # ... add antenna gain
P_0 = 10**(-10 / 10) * 1e-3                                                     # FPC parameter
alpha = 0.5                                                                     # Path-loss compensation factor

# Noise variance
noise_figure = 9                                                                # Noise figure in dB
N0 = -174                                                                       # PSD noise in dBm/Hz (AWGN)
noise_variance = BW * 10**(0.1 * noise_figure) * 10**(0.1 * N0) * 1e-3          # F*N0*B

# Pilot assignment
tau_c = 200                                                                     # Length of coherence interval in time/frequency samples
tau_p = int(np.ceil(K / 2))                                                     # Number of pilot symbols
Pilot_Set = PilotSetOrthogonal(tau_p)                                           # Set of orthogonal pilots
Phi, Assignment = PilotAssignmentRandom(K, tau_p, Pilot_Set)                    # Random pilot assignment
P_training = 100e-3                                                             # Users (UL) training power
P_training *= antenna_gain                                                      # ... add antenna gain
Eta_pilot = tau_p * P_training * np.ones(K)                                     # ... over tau_p sequences

# Pure CF (or fully-connected network) associations
M_ASS_CF = [list(range(0, M_CF)) for _ in range(K)]                             # All APs are associated to all users

# Data rates and specific absorption rates (SARs)
R_CF = np.zeros((K, N_power, N_channels))
R_UC = np.zeros((K, N_power, N_channels))
R_APC = np.zeros((K, N_power, N_channels))
R_MC = np.zeros((K, N_power, N_channels))

S_CF = np.zeros((K, N_power, N_channels))
S_UC = np.zeros((K, N_power, N_channels))
S_APC = np.zeros((K, N_power, N_channels))
S_MC = np.zeros((K, N_power, N_channels))

# Electromagnetic field (EMF) exposure UL constraints
b = 8                                                                           # SAR coefficient
EMF = 0.08                                                                      # Whole-body maximum SAR in W/kg

if constraint == 0:
    EMF = np.inf

# Monte Carlo iterations
for n in range(0, N_channels):
    print(f'Channel realization {n + 1} out of {N_channels}')

    # User and AP positions
    AP_pos = np.concatenate((D * np.random.rand(M_CF, 2), h_AP * np.ones((M_CF, 1))), axis=1)
    user_pos = np.concatenate((D * np.random.rand(K, 2), h_user * np.ones((K, 1))), axis=1)

    # 4 fixed BSs with N_AP antenna each and located at the center of the 4 quadrants of the deployment area.
    BS_pos = np.zeros((M_MC, 3))
    BS_pos[0, :] = [D / 4, D / 4, h_AP]
    BS_pos[1, :] = [3 * D / 4, D / 4, h_AP]
    BS_pos[2, :] = [3 * D / 4, 3 * D / 4, h_AP]
    BS_pos[3, :] = [D / 4, 3 * D / 4, h_AP]

    # Guarantee minimum distance
    user_pos = GuaranteeMinimumDistance(np.concatenate((AP_pos, BS_pos), axis=0), user_pos, D, M_CF + M_MC, K, d_min, wrapped)

    if n == 0:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(user_pos[:, 0], user_pos[:, 1], user_pos[:, 2], marker='x')
        ax.scatter(AP_pos[:, 0], AP_pos[:, 1], AP_pos[:, 2], marker='o')
        ax.scatter(BS_pos[:, 0], BS_pos[:, 1], BS_pos[:, 2], marker='s')
        ax.set_xlabel('X-axis', labelpad=10)
        ax.set_ylabel('Y-axis', labelpad=10)
        ax.set_title('Deployment Example', fontsize=15)
        ax.legend(['User', 'AP', 'BS'], loc='best')
        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        ax.minorticks_on()
        plt.show()

    # Scenarios CF-mMIMO
    # Channel and large-scale fading coefficients
    H, Beta, C = ChannelModel(f, AP_pos, user_pos, N_AP, M_CF, K, antenna_spacing, wavelength, wrapped, D, d_min, fading)

    # LMMSE estimation
    if CSI == 1:
        H_estimate = H
    elif CSI == 2:
        H_estimate = ChannelEstimation(N_AP, M_CF, K, tau_p, H, C, Phi, Eta_pilot, noise_variance)

    # MRC equalizer
    G = Filtering_UL(N_AP, M_CF, K, H_estimate)

    # I) Pure CF
    # Associations
    M_ASS = M_ASS_CF

    # Power control results
    R, S = PowerControl_UL(K, N_power, C, P_user, M_ASS, b, EMF, H, G, noise_variance, P_0, alpha)
    R_CF[:, :, n] = R
    S_CF[:, :, n] = S

    # II) UC
    # Associations
    _, M_ASS = UserAssociationUC(M_CF, K, Beta, N_APs_UC)

    # Power control results
    R, S = PowerControl_UL(K, N_power, C, P_user, M_ASS, b, EMF, H, G, noise_variance, P_0, alpha)
    R_UC[:, :, n] = R
    S_UC[:, :, n] = S

    # III) APC
    # Associations
    _, M_ASS = UserAssociationAPC(M_CF, K, Beta, N_users_APC)

    # Power control results
    R, S = PowerControl_UL(K, N_power, C, P_user, M_ASS, b, EMF, H, G, noise_variance, P_0, alpha)
    R_APC[:, :, n] = R
    S_APC[:, :, n] = S

    # Scenario MC-mMIMO
    # Channel and large-scale fading coefficients
    H, Beta, C = ChannelModel(f, BS_pos, user_pos, N_BS, M_MC, K, antenna_spacing, wavelength, wrapped, D, d_min, fading)

    # Channel estimation
    if CSI == 1:
        H_estimate = H
    elif CSI == 2:
        H_estimate = ChannelEstimation(N_BS, M_MC, K, tau_p, H, C, Phi, Eta_pilot, noise_variance)

    # MRC equalizer
    G = Filtering_UL(N_BS, M_MC, K, H_estimate)

    # User association MC-mMIMO
    _, M_ASS = UserAssociationUC(M_MC, K, Beta, 1)

    # Power control results
    R, S = PowerControl_UL(K, N_power, C, P_user, M_ASS, b, EMF, H, G, noise_variance, P_0, alpha)
    R_MC[:, :, n] = R
    S_MC[:, :, n] = S

# Convert rates from bps/Hz to bps
R_CF *= BW * (1 - tau_p / tau_c) / 2
R_UC *= BW * (1 - tau_p / tau_c) / 2
R_APC *= BW * (1 - tau_p / tau_c) / 2
R_MC *= BW * (1 - tau_p / tau_c) / 2

# Plot rate results
plt.figure()
markers = ["o","s"]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
for i in range(N_power):
      ind = 0
      aux_R = R_CF[:, i, :]
      ecdf = scipy.stats.ecdf(aux_R.ravel() / 1e6)
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 40))

for i in range(N_power):
      ind = 1
      aux_R = R_UC[:, i, :]
      ecdf = scipy.stats.ecdf(aux_R.ravel() / 1e6)
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 20))

for i in range(N_power):
      ind = 2
      aux_R = R_APC[:, i, :]
      ecdf = scipy.stats.ecdf(aux_R.ravel() / 1e6)
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 10))

for i in range(N_power):
      ind = 3
      aux_R = R_MC[:, i, :]
      ecdf = scipy.stats.ecdf(aux_R.ravel() / 1e6)
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 5))

plt.xlabel('Rate per user [Mbit/s]', fontsize=15)
plt.ylabel('CDF', fontsize=15)
plt.title('Throughput vs Power Control (UL)', fontsize=15)
plt.legend(['CF-mMIMO: UPC', 'CF-mMIMO: FPC', 'UC-CF-mMIMO: UPC', 'UC-CF-mMIMO: FPC', 'APC-CF-mMIMO: UPC',
            'APC-CF-mMIMO: FPC', 'MC-mMIMO: UPC', 'MC-mMIMO: FPC'], loc='best', fontsize=15)
plt.grid()
plt.show()

# Plot SAR results
plt.figure()
for i in range(N_power):
      ind = 0
      aux_S = S_CF[:, i, :]
      ecdf = scipy.stats.ecdf(aux_S.ravel())
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      if len(x) == 1:                                                           # UPC might lead to single values
          plt.plot(x*np.ones(K * N_channels), np.linspace(0, 1, K * N_channels))
      else:
          plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 40))

for i in range(N_power):
      ind = 1
      aux_S = S_UC[:, i, :]
      ecdf = scipy.stats.ecdf(aux_S.ravel())
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      if len(x) == 1:                                                           # UPC might lead to single values
          plt.plot(x*np.ones(K * N_channels), np.linspace(0, 1, K * N_channels))
      else:
          plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 20))

for i in range(N_power):
      ind = 2
      aux_S = S_APC[:, i, :]
      ecdf = scipy.stats.ecdf(aux_S.ravel())
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      if len(x) == 1:                                                           # UPC might lead to single values
          plt.plot(x*np.ones(K * N_channels), np.linspace(0, 1, K * N_channels))
      else:
          plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 10))

for i in range(N_power):
      ind = 3
      aux_S = S_MC[:, i, :]
      ecdf = scipy.stats.ecdf(aux_S.ravel())
      x = ecdf.cdf.quantiles
      y = ecdf.cdf.probabilities
      if len(x) == 1:                                                           # UPC might lead to single values
          plt.plot(x*np.ones(K * N_channels), np.linspace(0, 1, K * N_channels))
      else:
          plt.plot(x, y)
      plt.gca().lines[i + 2*ind].set_color(colors[ind])
      plt.gca().lines[i + 2*ind].set_marker(markers[i])
      plt.gca().lines[i + 2*ind].set_markevery(int(K * N_channels / 5))

plt.xlabel('SAR per user [W/kg]', fontsize=15)
plt.ylabel('CDF', fontsize=15)
plt.title('Exposure vs Power Control (UL)', fontsize=15)
plt.legend(['CF-mMIMO: UPC', 'CF-mMIMO: FPC', 'UC-CF-mMIMO: UPC', 'UC-CF-mMIMO: FPC', 'APC-CF-mMIMO: UPC',
            'APC-CF-mMIMO: FPC', 'MC-mMIMO: UPC', 'MC-mMIMO: FPC'], loc='best', fontsize=15)
plt.xscale('log')
plt.grid()
plt.show()